In [1]:
import tensorflow as tf
from tensorflow import keras
import kerastuner
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os
import json
import datetime

plt.rcParams["figure.figsize"] = (20, 5)

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [2]:
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
'''
log_dir = "logs/fit/" + timestamp
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
'''
version_dir = "version/" + timestamp 

os.makedirs(version_dir)
timestamp

'20200905-093844'

In [3]:
dataset_name = "SEG_2"

In [4]:
dataset = np.genfromtxt("data/{}_train_set.csv".format(dataset_name), delimiter="\n", dtype=np.float32) #np.int64
dataset

array([0., 0., 0., ..., 1., 3., 1.], dtype=float32)

In [5]:
word_index = np.genfromtxt("data/word_index.csv", delimiter="\n", dtype=np.int64)
vocab_size = len(word_index)
vocab_size

14882

In [6]:
param_list = dict()

param_list["PAST_HISTORY"] = 16
param_list["FUTURE_TARGET"] = 8
param_list["BATCH_SIZE"] = 128
param_list["EPOCHS"] = 100
param_list["BUFFER_SIZE"] = 200000
param_list["VOCAB_SIZE"] = vocab_size
param_list["LEARNING_RATE"] = 0.01
param_list["NUM_1_NEURONS"] = 177
param_list["NUM_2_NEURONS"] = 177
param_list["DROPOUT_1"] = 0.1
param_list["DROPOUT_2"] = 0.2

In [7]:
def generate_timeseries(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, n_feature)
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        #data.append(dataset[indices])
        labels.append(np.reshape(dataset[i:i+target_size], (target_size, 1)))
        #labels.append(dataset[i:i+target_size])
    return np.array(data), np.array(labels)

In [8]:
x_train, y_train = generate_timeseries(dataset, 0, None, param_list["PAST_HISTORY"], param_list["FUTURE_TARGET"])
x_train.shape, y_train.shape

((14858, 16, 1), (14858, 8, 1))

In [9]:
x_train[10], y_train[10]

(array([[  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [933.],
        [  0.],
        [  0.],
        [  0.],
        [  0.]], dtype=float32),
 array([[ 0.],
        [ 0.],
        [48.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.]], dtype=float32))

model = keras.models.Sequential()
model.add(keras.layers.Bidirectional(tf.keras.layers.LSTM(param_list["NUM_1_NEURONS"])))
model.add(keras.layers.Dropout(param_list["DROPOUT_1"]))
model.add(keras.layers.RepeatVector(param_list["FUTURE_TARGET"]))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(param_list["NUM_2_NEURONS"], return_sequences=True)))
model.add(keras.layers.Dropout(param_list["DROPOUT_2"]))
model.add(keras.layers.TimeDistributed(keras.layers.Dense(param_list["VOCAB_SIZE"], activation='softmax')))
model.compile(optimizer=keras.optimizers.Adam(param_list["LEARNING_RATE"]), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model_history = model.fit(x_train, y_train, batch_size=param_list["BATCH_SIZE"], validation_split=0.2, epochs=param_list["EPOCHS"])

In [41]:
def get_model_deep(shape=(param_list["PAST_HISTORY"], 1)):
    inp = keras.layers.Input(shape)
    #x = keras.layers.Dense(16)(inp)
    #x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True))(inp)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.TimeDistributed(keras.layers.Dense(64))(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True))(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.TimeDistributed(keras.layers.Dense(param_list["FUTURE_TARGET"]))(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.TimeDistributed(keras.layers.Dense(param_list["VOCAB_SIZE"], activation='softmax'))(x)

    model = keras.Model(inp, x)
    return model

In [42]:
model = get_model_deep()
model.compile(optimizer=keras.optimizers.Nadam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #
input_9 (InputLayer)         [(None, 16, 1)]           0
_________________________________________________________________
bidirectional_21 (Bidirectio (None, 16, 128)           33792
_________________________________________________________________
batch_normalization_42 (Batc (None, 16, 128)           512
_________________________________________________________________
time_distributed_25 (TimeDis (None, 16, 64)            8256
_________________________________________________________________
batch_normalization_43 (Batc (None, 16, 64)            256
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 16, 256)           197632
_________________________________________________________________
batch_normalization_44 (Batc (None, 16, 256)           1024
________________________________________

In [33]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_delta=0.01)

In [34]:
model_history = model.fit(x_train, y_train, 
    batch_size=param_list["BATCH_SIZE"], validation_split=0.2, epochs=param_list["EPOCHS"], 
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/100


ValueError: in user code:

    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:543 train_step  **
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\compile_utils.py:411 update_st

result = model.predict(x_train[10000].reshape(1, -1, 1))
result, result.shape

model.save("version/{}/model.h5".format(timestamp))